In [1]:
class counter:
    GLOBAL_REF_TRACKER = {}
    def __init__(self):
        obj_id = id(self)
        counter.GLOBAL_REF_TRACKER[obj_id] = 1
    def add_ref(self):
        obj_id = id(self)
        counter.GLOBAL_REF_TRACKER[obj_id] += 1
    def release_ref(self):
        obj_id = id(self)
        counter.GLOBAL_REF_TRACKER[obj_id] -= 1
        if counter.GLOBAL_REF_TRACKER[obj_id] == 0 :
            print(f"{obj_id} : 객체 소멸")
            del counter.GLOBAL_REF_TRACKER[obj_id]
    def copy(self):
        return counter()

In [4]:
class MemoryObject:
    # 클래스 변수: 전체 객체 관리
    MEM_TRACKER = {}

    def __init__(self, name):
        self.name = name  # 객체 이름
        obj_id = id(self)
        # 객체 등록
        MemoryObject.MEM_TRACKER[obj_id] = {
            "count": 1,
            "children": [],
            "name": self.name
        }

    def add_ref(self):
        obj_id = id(self)
        MemoryObject.MEM_TRACKER[obj_id]["count"] += 1  # 참조 수 증가

    def release_ref(self):
        obj_id = id(self)
        MemoryObject.MEM_TRACKER[obj_id]["count"] -= 1  # 참조 수 감소

        # 참조 수가 0이면 소멸
        if MemoryObject.MEM_TRACKER[obj_id]["count"] == 0:
            print(f"{obj_id} : 객체 소멸")
            del MemoryObject.MEM_TRACKER[obj_id]  # MEM_TRACKER에서 삭제

    def add_child(self, child):
        obj_id = id(self)
        child_id = id(child)
        # 자식 객체 추가 및 참조 수 증가
        MemoryObject.MEM_TRACKER[obj_id]["children"].append(child)
        child.add_ref()
        
    def remove_child(self,clild):
        obj_id = id(self)
        child_id = id(child)
        # 자식 객체가 children 목록에 있는지 확인
        if child in MemoryObject.MEM_TRACKER[obj_id]["children"]:
            # 참조 수 감소 및 자식 제거
            MemoryObject.MEM_TRACKER[obj_id]["children"].remove(child)
            child.release_ref()
        else:
            # 자식이 없을 때 처리 (선택 사항)
            print(f"{child_id} : 자식 객체가 존재하지 않아 삭제할 수 없습니다.")
    
    @classmethod
    def show_tracker(cls):
        # 현재 MEM_TRACKER 상태 출력
        for obj_id, info in cls.MEM_TRACKER.items():
            print(f"ID: {obj_id}, Info: {info}")
    
if __name__ == "__main__":
    print("\n--- [ 순환 참조 생성 및 기본 레퍼런스 카운트 관리 ] ---")
    # 두 객체 생성
    objA = MemoryObject("A")
    objB = MemoryObject("B")

    # 서로를 자식으로 추가하여 순환 참조 형성
    objA.add_child(objB)  # A -> B (B의 카운트 +1)
    objB.add_child(objA)  # B -> A (A의 카운트 +1)

    # 현재 MEM_TRACKER 상태 출력
    print("\nMEM_TRACKER 상태 (순환 참조 형성 후):")
    for obj_id, info in MemoryObject.MEM_TRACKER.items():
        print(f" id: {obj_id}, name: {info['name']}, count: {info['count']}")

    # 외부(root) 참조 제거: 여기서는 부모 자체를 None 처리
    # (하지만 내부 순환 참조 때문에 카운트는 2로 남음)
    objA.release_ref()  # 외부 참조 제거: A의 count 감소
    objB.release_ref()  # B의 count 감소 (원래 2 -> 1)

    print("\n외부 참조 제거 후 MEM_TRACKER 상태:")
    for obj_id, info in MemoryObject.MEM_TRACKER.items():
        print(f" id: {obj_id}, name: {info['name']}, count: {info['count']}")
    print("\n※ 단순 release_ref만으로는 서로 참조하는 순환 구조 때문에 count가 0이 되지 않음.")



--- [ 순환 참조 생성 및 기본 레퍼런스 카운트 관리 ] ---

MEM_TRACKER 상태 (순환 참조 형성 후):
 id: 4534667216, name: A, count: 2
 id: 4534664720, name: B, count: 2

외부 참조 제거 후 MEM_TRACKER 상태:
 id: 4534667216, name: A, count: 1
 id: 4534664720, name: B, count: 1

※ 단순 release_ref만으로는 서로 참조하는 순환 구조 때문에 count가 0이 되지 않음.
